In [3]:
%pip install kaggle
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install seaborn



Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   -- ------------------------------------- 0.8/12.6 MB 4.8 MB/s eta 0:00:03
   ----- ---------------------------------- 1.8/12.6 MB 4.6 MB/s eta 0:00:03
   --------- ------------------------------ 2.9/12.6 MB 4.5 MB/s eta 0:00:03
   ----------- ---------------------------- 3.7/12.6 MB 4.5 MB/s eta 0:00:02
   -------------- ------------------------- 4.7/12.6 MB 4.5 MB/s eta 0:00:02
   ------------------ --------------------- 5.8/12.6 MB 4.5 MB/s eta 0:00:02
   -------------------- ------------------- 6.6/12.6 MB 4.5 MB/s eta 0:00:02
   ------------------------ --------------- 7.6/12.6 MB 4.5 MB/s eta 0:00:02
   ------------------------- -------------- 8.1/12.6 MB 4.3 MB/s eta 0:00:02
   ---------------------------- ----------- 8.9/12.6 MB 4.2 MB/s eta 0:00:01
   ----------------------------- ---------- 9.2/12.6 MB 4.0 MB/s eta 0:00:01
   --------------

In [5]:
%pip install scikit-learn

   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.1 MB 4.8 MB/s eta 0:00:03
   ---- ----------------------------------- 1.3/11.1 MB 2.8 MB/s eta 0:00:04
   ------- -------------------------------- 2.1/11.1 MB 3.5 MB/s eta 0:00:03
   ---------- ----------------------------- 2.9/11.1 MB 3.2 MB/s eta 0:00:03
   ------------- -------------------------- 3.7/11.1 MB 3.5 MB/s eta 0:00:03
   ---------------- ----------------------- 4.7/11.1 MB 3.7 MB/s eta 0:00:02
   ------------------- -------------------- 5.5/11.1 MB 3.8 MB/s eta 0:00:02
   ----------------------- ---------------- 6.6/11.1 MB 3.9 MB/s eta 0:00:02
   --------------------------- ------------ 7.6/11.1 MB 3.9 MB/s eta 0:00:01
   ------------------------------- -------- 8.7/11.1 MB 4.0 MB/s eta 0:00:01
   ---------------------------------- ----- 9.7/11.1 MB 4.1 MB/s eta 0:00:01
   ------------------------------------- -- 10.5/11.1 MB 4.1 MB/s eta 0:00:01
   --

Imports

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import PolynomialFeatures

In [7]:
!kaggle datasets list -s "movie"

ref                                                             title                                            size  lastUpdated                 downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  -----------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
danielgrijalvas/movies                                          Movie Industry                                 433664  2021-07-23 19:48:26.173000          57266        455  1.0              
tmdb/tmdb-movie-metadata                                        TMDB 5000 Movie Dataset                       9317430  2017-09-28 01:09:12.897000         433913       4009  0.8235294        
jrobischon/wikipedia-movie-plots                                Wikipedia Movie Plots                        31376869  2018-10-15 19:59:54.650000          28871        505  0.88235295       
shivamb/netflix-shows                        

In [8]:
!kaggle datasets download tmdb/tmdb-movie-metadata

Dataset URL: https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata
License(s): other




  0%|          | 0.00/8.89M [00:00<?, ?B/s]
100%|██████████| 8.89M/8.89M [00:00<00:00, 462MB/s]


 Unzip the file to access contents and list files

In [10]:
import zipfile
import os

with zipfile.ZipFile("tmdb-movie-metadata.zip", "r") as zip_ref:
    zip_ref.extractall("tmdb_movie_data")

os.listdir("tmdb_movie_data")

['tmdb_5000_credits.csv', 'tmdb_5000_movies.csv']

Read CSV files into pandas dataframe

In [8]:
path = "tmdb_movie_data/tmdb_5000_movies.csv"
df = pd.read_csv(path)
df.head(10)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
5,258000000,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""na...",http://www.sonypictures.com/movies/spider-man3/,559,"[{""id"": 851, ""name"": ""dual identity""}, {""id"": ...",en,Spider-Man 3,The seemingly invincible Spider-Man goes up ag...,115.699814,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-01,890871626,139.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,The battle within.,Spider-Man 3,5.9,3576
6,260000000,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751...",http://disney.go.com/disneypictures/tangled/,38757,"[{""id"": 1562, ""name"": ""hostage""}, {""id"": 2343,...",en,Tangled,When the kingdom's most wanted-and most charmi...,48.681969,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2010-11-24,591794936,100.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,They're taking adventure to new lengths.,Tangled,7.4,3330
7,280000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://marvel.com/movies/movie/193/avenge